In [1]:
import pandas as pd
from datetime import datetime
from unidecode import unidecode

from sys import exit

def trim_overspace(text):
	parts = filter(lambda x: len(x) > 0,text.split(" "))
	return " ".join(parts)

def normalize_text(text):
    #.replace("'"," ") alguns municipios tem ' e as macros maravilhosas do excel vão falhar
	x = str(text).replace(".","").replace("\n","").replace(",","").upper()
	x = trim_overspace(x)
	x = unidecode(x)

	if x == 'nan' or len(x) == 0 or x == '0' or x == 'nao_informado':
		return ''
	else:
		return x
    
def normalize_labels(text):
	x = str(text).replace("'"," ").replace(".","").replace("\n","").replace(",","").lower()
	x = trim_overspace(x).replace(" ","_")
	x = unidecode(x)

	if x == 'nan' or len(x) == 0 or x == '0' or x == 'nao_informado':
		return ''
	else:
		return x
    
def normalize_number(num,cast=int,error='fill',fill='0'):
	try:
		return cast(num)
	except ValueError:
		if error == 'raise':
			raise Exception(ValueError)
		elif error == 'fill':
			return normalize_number(fill,cast,'raise')

In [2]:
casos_raw = pd.read_excel('novos_casos_raw.xlsx', dtype={'rs':str})
casos_raw.dtypes

id                           int64
data_notificacao    datetime64[ns]
paciente                    object
sexo                        object
idade                        int64
mun_resid                   object
mun_atend                   object
rs                          object
nome_exame                  object
data_liberacao      datetime64[ns]
data_1o_sintomas    datetime64[ns]
dtype: object

In [3]:
casos_raw['hash'] = casos_raw.apply(lambda row: (row['paciente'] + str(row['idade']) + row['mun_resid']).replace(" ",""), axis=1)
casos_raw['hash_less'] = casos_raw.apply(lambda row: (row['paciente'] + str(row['idade']-1) + row['mun_resid']).replace(" ",""), axis=1)
casos_raw['hash_more'] = casos_raw.apply(lambda row: (row['paciente'] + str(row['idade']+1) + row['mun_resid']).replace(" ",""), axis=1)

In [4]:
casos_raw = casos_raw.sort_values(by='paciente')

In [5]:
print(f"casos novos: {casos_raw.shape[0]}")
print(f"casos novos duplicados: {casos_raw.loc[casos_raw.duplicated(subset='hash')].shape[0]}")
casos_raw = casos_raw.drop_duplicates(subset='hash')
print(f"casos novos sem duplicados: {casos_raw.shape[0]}")

casos novos: 3622
casos novos duplicados: 1
casos novos sem duplicados: 3621


In [6]:
casos_confirmados = pd.read_excel('Casos confirmados PR.xlsx', 'Casos confirmados', usecols='D:M')
casos_confirmados.columns = [ normalize_labels(x) for x in casos_confirmados.columns]

In [7]:
casos_confirmados['nome'] = casos_confirmados['nome'].apply(normalize_text)
casos_confirmados['idade'] = casos_confirmados['idade'].apply(normalize_number)
casos_confirmados['mun_resid'] = casos_confirmados['mun_resid'].apply(normalize_text)
casos_confirmados['mun_atend'] = casos_confirmados['mun_atend'].apply(normalize_text)
casos_confirmados['hash'] = casos_confirmados.apply(lambda row: (row['nome'] + str(row['idade']) + row['mun_resid']).replace(" ",""), axis=1)
casos_confirmados.dtypes

nome                   object
sexo                   object
idade                   int64
mun_resid              object
mun_atend              object
rs                    float64
laboratorio            object
dt_diag                object
comunicacao    datetime64[ns]
is                     object
hash                   object
dtype: object

In [8]:
index_casos_duplicados = casos_raw.loc[casos_raw['hash'].isin(casos_confirmados['hash'])].index.to_list()
print(f"casos já em casos com a mesma idade: {len(index_casos_duplicados)}")
index_casos_duplicados_idade_less = casos_raw.loc[casos_raw['hash_less'].isin(casos_confirmados['hash'])].index.to_list()
print(f"casos já em casos com a idade - 1: {len(index_casos_duplicados_idade_less)}")
index_casos_duplicados_idade_more = casos_raw.loc[casos_raw['hash_more'].isin(casos_confirmados['hash'])].index.to_list()
print(f"casos já em casos com a idade + 1: {len(index_casos_duplicados_idade_more)}")
print(f"dentre os quais {len(set(index_casos_duplicados_idade_more).intersection(index_casos_duplicados)) + len(set(index_casos_duplicados_idade_less).intersection(index_casos_duplicados))} são casos em comum, o que leva a crer que estão duplicados na planilha já com idade a mais ou idade a menos")
index_duplicados = list(set(index_casos_duplicados + index_casos_duplicados_idade_less + index_casos_duplicados_idade_more))
print(f"sendo assim, {len(index_duplicados)} casos que já se encontram na planilha serão removidos")

casos já em casos com a mesma idade: 922
casos já em casos com a idade - 1: 26
casos já em casos com a idade + 1: 1
dentre os quais 0 são casos em comum, o que leva a crer que estão duplicados na planilha já com idade a mais ou idade a menos
sendo assim, 949 casos que já se encontram na planilha serão removidos


In [9]:
print(f"então, de {len(casos_raw)} casos baixados hoje  {len(casos_raw)-len(index_duplicados)} serão adicionados")
casos_raw = casos_raw.drop(index=index_duplicados)
casos_raw.shape

então, de 3621 casos baixados hoje  2672 serão adicionados


(2672, 14)

In [10]:
casos_raw['data_com'] = [ "" ] * len(casos_raw)
casos_raw.columns

Index(['id', 'data_notificacao', 'paciente', 'sexo', 'idade', 'mun_resid',
       'mun_atend', 'rs', 'nome_exame', 'data_liberacao', 'data_1o_sintomas',
       'hash', 'hash_less', 'hash_more', 'data_com'],
      dtype='object')

In [11]:
novos_casos = casos_raw[['paciente','sexo','idade','mun_resid', 'mun_atend', 'rs', 'nome_exame','data_notificacao','data_liberacao','data_com','data_1o_sintomas']]
novos_casos.to_excel('novos_casos.xlsx', index=False)